## Read and save all Visium objects

In [1]:
import sys 
import os
from datetime import datetime
date = datetime.now().strftime("%Y-%m-%d")
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
import hdf5plugin

# Import key analysis packages
import cell2location
import scvi


# Define repo path and add it to sys path (allows to access scripts and metadata from repo)
repo_path = '/nfs/team205/vk8/projects/thymus_ageing_atlas/Spatial_analyses'
sys.path.insert(1, repo_path) 
sys.path.insert(2, '/nfs/team205/vk8/projects/thymus_ageing_atlas/General_analysis/scripts')


In [2]:
# Define paths for plot and path dirs
plot_path = os.path.join(repo_path, 'plots')
data_path = os.path.join(repo_path, 'data')
results_path = os.path.join(repo_path, 'results')
model_path = os.path.join(repo_path, 'models')
general_data_path = '/nfs/team205/vk8/projects/thymus_ageing_atlas/General_analysis/data'

In [3]:
# Set pandas display options
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to show output from all the lines in a cells
pd.set_option('display.max_column',None) # display all the columns in pandas
pd.options.display.max_rows = 100

In [4]:
%load_ext autoreload
%autoreload 2
# Import custom scripts
from utils import get_latest_version,update_obs,freq_by_donor
from anno_levels import get_ct_levels, get_ct_palette, age_group_levels, age_group_palette
#from plotting.utils import plot_grouped_boxplot, calc_figsize

In [5]:
# Set plot formatting
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager
from matplotlib import rcParams
font_manager.fontManager.addfont("/nfs/team205/ny1/ThymusSpatialAtlas/software/Arial.ttf")
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text

In [5]:
sc.settings.set_figure_params(dpi = 80, color_map = 'RdPu')
#plt.style.use('/nfs/team205/vk8/projects/thymus_ageing_atlas/General_analysis/scripts/plotting/thyAgeing.mplstyle')

## Load Visium objects

In [6]:
meta_df = pd.read_excel(f'{data_path}/ThyAge_spatial_metadata_v2_2025-09-02.xlsx')

In [9]:
samples = meta_df.loc[meta_df['chemistry_simple']!='Xenium', 'SampleID'].tolist()
len(samples)

38

samples_ffpe = meta_df.loc[meta_df['chemistry_simple']=='Visium_FFPE', 'SampleID'].tolist()
samples_oct = meta_df.loc[meta_df['chemistry_simple']=='Visium_OCT', 'SampleID'].tolist()

sample1 = samples[0]
path1 = meta_df.loc[meta_df['SampleID']==sample1, 'Path'].values[0]
adata1 = sc.read_visium(f'{path1}/{sample1}/outs/')
list(adata1.uns['spatial'].keys())[0]
sample1 = samples_oct[0]
path1 = meta_df.loc[meta_df['SampleID']==sample1, 'Path'].values[0]
adata1 = sc.read_visium(f'{path1}/{sample1}/')
list(adata1.uns['spatial'].keys())[0]

old_sp_key = list(adata1.uns['spatial'].keys())[0]
new_sp_key = meta_df.loc[meta_df['SampleID']==sample1, 'Sample_hr'].values[0]
adata1.uns['spatial'][new_sp_key] = adata1.uns['spatial'].pop(old_sp_key)

In [10]:
# load the raw slides
slides = []
# iterate over the dataframe rows
for sample in samples:
    # load the anndata object from the path
    path = meta_df.loc[meta_df['SampleID']==sample, 'Path'].values[0]
    if meta_df.loc[meta_df['SampleID']==sample, 'chemistry_simple'].values[0] == 'Visium_FFPE':
        adata = sc.read_visium(f'{path}/{sample}/outs/')
    else:
        adata = sc.read_visium(f'{path}/{sample}/')
    adata.obs['SampleID'] = sample
    adata.var['SYMBOL'] = adata.var_names
    adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)
    adata.var_names_make_unique()
    # calculate QC metrics
    sc.pp.calculate_qc_metrics(adata, percent_top = None, inplace=True)
    adata.var['mt'] = [gene.startswith('MT-') for gene in adata.var['SYMBOL']]
    adata.obs['percent_mito'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']
    # add sample name to obs names
    adata.obs_names = adata.obs["SampleID"] + '-' + adata.obs_names
    adata.obs.index.name = 'spot_id'
    # rename the spatial key to match sample_hr
    old_sp_key = list(adata.uns['spatial'].keys())[0]
    adata.uns['spatial'][sample] = adata.uns['spatial'].pop(old_sp_key)
    print(f'{path}/{sample}/ has been read')
    slides.append(adata)

adata = slides[0].concatenate(
        slides[1:],
        uns_merge="unique",
        index_unique=None
    )



/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/lustre/scratch126/cellgen/teichmann/vk8/ThyAge_rawdata/Visium/THY81/ has been read


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/nfs/team205/vk8/irods_data/09_thymus/visium/all_thymus_visium/raw_data2/TA11486162/ has been read


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


...

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/nfs/team205/vk8/irods_data/09_thymus/visium/all_thymus_visium/raw_data2/WSSS_THYst9383371/ has been read


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/nfs/team205/vk8/irods_data/09_thymus/visium/all_thymus_visium/raw_data2/WSSS_THYst9383372/ has been read


/tmp/ipykernel_4082179/360833051.py:28: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = slides[0].concatenate(


In [ ]:
adata.var = adata.var[['ENSEMBL', 'feature_types', 'genome', 'SYMBOL', 'mt']].copy()
new_obs = adata.obs.merge(meta_df, on = 'SampleID', how = 'left').set_index(adata.obs_names)
adata.obs = new_obs.copy()
adata.obs.drop(columns = 'batch', inplace=True)

In [38]:
new_meta = pd.read_excel(f'{data_path}/ThyAge_spatial_metadata_v3_2025-09-03.xlsx')

In [49]:
new_meta.rename(columns={'section_thickness (um)':'section_thickness(um)'}, inplace=True)

In [50]:
cols2add = list(np.setdiff1d(new_meta.columns.tolist(), adata.obs.columns.to_list()))

In [52]:
update_obs(adata, new_obs = new_meta, on = 'SampleID', add_cols = cols2add)

No columns specified to update. Automatically determining columns to update...
Updating columns: ['Batch', 'Block_type', 'Number of genes', 'Path', 'Probeset', 'QC', 'RIN/DV200', 'Sample', 'Sample_hr', 'Sex', 'Source', 'Spaceranger', 'Study', 'Study_name', 'age_group', 'age_misc', 'age_months', 'age_numeric', 'annotation_path', 'annotation_version', 'annotator', 'chemistry_detail', 'chemistry_simple', 'donor', 'donor_type', 'published', 'reference', 'Block', 'Position', 'RawData', 'Sequencer', 'SlideID', 'permebialisation(min)', 'section_thickness(um)']
On columns: SampleID


In [58]:
col_order = ['in_tissue', 'array_row', 'array_col', 'SampleID', 'n_genes_by_counts',
       'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts',
       'percent_mito','Sample', 'Sample_hr', 'Path', 'donor', 'donor_type',
       'age_group', 'age_misc', 'age_numeric', 'age_months', 'Sex', 'Source',
       'SlideID', 'Position', 'section_thickness(um)', 'permebialisation(min)',
       'RIN/DV200', 'Block', 'Block_type', 'chemistry_simple',
       'chemistry_detail', 'Probeset', 'Number of genes', 'Study',
       'Study_name', 'published', 'QC', 'Batch', 'RawData', 'Sequencer',
       'Spaceranger', 'reference', 'annotation_path', 'annotation_version',
       'annotator']

In [59]:
len(col_order)
len(adata.obs.columns)

43

43

In [61]:
adata.obs = adata.obs[col_order].copy()

In [63]:
for col in adata.obs.columns:
    if (adata.obs[col].dtype.name == 'object') or (adata.obs[col].dtype.name == 'category'):
        adata.obs[col] = adata.obs[col].astype(str)

In [64]:
adata.write_h5ad(
        f'{data_path}/visium/thyAgeing_Visium_OCT_FFPE_raw_{date}.zarr',
        compression=hdf5plugin.FILTERS["zstd"],
        compression_opts=hdf5plugin.Zstd(clevel=5).filter_options)

In [ ]:
# load the raw slides
slides_ffpe = []
# iterate over the dataframe rows
for sample in samples_ffpe:
    # load the anndata object from the path
    path = meta_df.loc[meta_df['SampleID']==sample, 'Path'].values[0]
    adata = sc.read_visium(f'{path}/{sample}/outs/')
    adata.obs['SampleID'] = sample
    adata.var['SYMBOL'] = adata.var_names
    adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)
    adata.var_names_make_unique()
    # calculate QC metrics
    sc.pp.calculate_qc_metrics(adata, percent_top = None, inplace=True)
    adata.var['mt'] = [gene.startswith('MT-') for gene in adata.var['SYMBOL']]
    adata.obs['percent_mito'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']
    # add sample name to obs names
    adata.obs_names = adata.obs["SampleID"] + '-' + adata.obs_names
    adata.obs.index.name = 'spot_id'
    # rename the spatial key to match sample_hr
    old_sp_key = list(adata.uns['spatial'].keys())[0]
    new_sp_key = meta_df.loc[meta_df['SampleID']==sample, 'Sample_hr'].values[0]
    adata.uns['spatial'][new_sp_key] = adata.uns['spatial'].pop(old_sp_key)
    print(f'{path} has been read')
    slides_ffpe.append(adata)

adata_ffpe = slides_ffpe[0].concatenate(
        slides_ffpe[1:],
        uns_merge="unique",
        index_unique=None
    )

adata_ffpe.obs.head()


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/lustre/scratch126/cellgen/teichmann/vk8/ThyAge_rawdata/Visium has been read


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/lustre/scratch126/cellgen/teichmann/vk8/ThyAge_rawdata/Visium has been read


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


...

/lustre/scratch126/cellgen/teichmann/vk8/ThyAge_rawdata/Visium has been read


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/lustre/scratch126/cellgen/teichmann/vk8/ThyAge_rawdata/Visium has been read


/tmp/ipykernel_4068864/3587660486.py:26: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_ffpe = slides_ffpe[0].concatenate(


,in_tissue,array_row,array_col,SampleID,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,percent_mito,batch
spot_id,,,,,,,,,,
THY81-AACAATGGAACCACAT-1,1,54,32,THY81,6356,8.757312,10713.0,9.279306,0.022403,0
THY81-AACACCATTCGCATAC-1,1,53,139,THY81,2675,7.892078,3564.0,8.178920,0.018519,0
THY81-AACACCGAGCTTGGAT-1,1,56,24,THY81,2698,7.900637,3673.0,8.209036,0.059080,0
THY81-AACACGACAACGGAGT-1,1,71,203,THY81,4793,8.475120,6417.0,8.766862,0.031323,0
THY81-AACACGCAGATAACAA-1,1,56,142,THY81,4823,8.481359,8192.0,9.011035,0.015137,0


In [11]:
adata_ffpe.obs

,in_tissue,array_row,array_col,SampleID,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,percent_mito,batch
spot_id,,,,,,,,,,
THY81-AACAATGGAACCACAT-1,1,54,32,THY81,6356,8.757312,10713.0,9.279306,0.022403,0
THY81-AACACCATTCGCATAC-1,1,53,139,THY81,2675,7.892078,3564.0,8.178920,0.018519,0
THY81-AACACCGAGCTTGGAT-1,1,56,24,THY81,2698,7.900637,3673.0,8.209036,0.059080,0
THY81-AACACGACAACGGAGT-1,1,71,203,THY81,4793,8.475120,6417.0,8.766862,0.031323,0
THY81-AACACGCAGATAACAA-1,1,56,142,THY81,4823,8.481359,8192.0,9.011035,0.015137,0
...,...,...,...,...,...,...,...,...,...,...
THY162-TGTTGGTGATTAGGTA-1,1,108,22,THY162,6659,8.803875,14933.0,9.611396,0.010179,5
THY162-TGTTGGTGCGCACGAG-1,1,24,108,THY162,4934,8.504108,8868.0,9.090318,0.008119,5
THY162-TGTTGGTGCGCTTCGC-1,1,37,127,THY162,4425,8.395252,7421.0,8.912204,0.009163,5


In [12]:
# load the raw slides
slides_oct = []
# iterate over the dataframe rows
for sample in samples_oct:
    # load the anndata object from the path
    path = meta_df.loc[meta_df['SampleID']==sample, 'Path'].values[0]
    adata = sc.read_visium(f'{path}/{sample}/')
    adata.obs['SampleID'] = sample
    adata.var['SYMBOL'] = adata.var_names
    adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)
    adata.var_names_make_unique()
    # calculate QC metrics
    sc.pp.calculate_qc_metrics(adata, percent_top = None, inplace=True)
    adata.var['mt'] = [gene.startswith('MT-') for gene in adata.var['SYMBOL']]
    adata.obs['percent_mito'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']
    # add sample name to obs names
    adata.obs_names = adata.obs["SampleID"] + '-' + adata.obs_names
    adata.obs.index.name = 'spot_id'
    # rename the spatial key to match sample_hr
    old_sp_key = list(adata.uns['spatial'].keys())[0]
    new_sp_key = meta_df.loc[meta_df['SampleID']==sample, 'Sample_hr'].values[0]
    adata.uns['spatial'][new_sp_key] = adata.uns['spatial'].pop(old_sp_key)
    print(f'{path}/{sample}/ has been read')
    slides_oct.append(adata)

adata_oct = slides_oct[0].concatenate(
        slides_oct[1:],
        uns_merge="unique",
        index_unique=None
    )

adata_oct.obs.head()


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/nfs/team205/vk8/irods_data/09_thymus/visium/all_thymus_visium/raw_data2/TA11486162/ has been read


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/nfs/team205/vk8/irods_data/09_thymus/visium/all_thymus_visium/raw_data2/TA11556493/ has been read


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


...

/nfs/team205/vk8/irods_data/09_thymus/visium/all_thymus_visium/raw_data2/WSSS_THYst9383371/ has been read


/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/nfs/team205/vk8/irods_data/09_thymus/visium/all_thymus_visium/raw_data2/WSSS_THYst9383372/ has been read


/tmp/ipykernel_4068864/219158057.py:26: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_oct = slides_oct[0].concatenate(


,in_tissue,array_row,array_col,SampleID,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,percent_mito,batch
spot_id,,,,,,,,,,
TA11486162-AAACAAGTATCTCCCA-1,1.0,50.0,102.0,TA11486162,750,6.621406,1029.0,6.937314,0.024295,0
TA11486162-AAACAATCTACTAGCA-1,1.0,3.0,43.0,TA11486162,1061,6.967909,1562.0,7.354362,0.018566,0
TA11486162-AAACACCAATAACTGC-1,1.0,59.0,19.0,TA11486162,1849,7.522941,3458.0,8.148735,0.043088,0
TA11486162-AAACAGAGCGACTCCT-1,1.0,14.0,94.0,TA11486162,433,6.073045,491.0,6.198479,0.010183,0
TA11486162-AAACAGGGTCTATATT-1,1.0,47.0,13.0,TA11486162,2110,7.654917,4043.0,8.304990,0.031165,0
